# Acquisitions (1 hour)
1. Communicate with the chip
2. Get a trace on the scope / Power analysis on AES
3. Get a set of (traces/plaintexts) to be attacked later

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt 
import time
from Crypto.Cipher import AES
#purely optional (for best visualisations of long loops)
from tqdm.notebook import tnrange


In [ ]:
# connection to the chip / set up chip specific parameters
SCOPETYPE='OPENADC'
PLATFORM='CWLITEARM'
CRYPTO_TARGET='TINYAES128C' 
CW_PATH = "../../chipwhisperer/jupyter/courses/sca101/"
setup = CW_PATH + "../../Setup_Scripts/Setup_Generic.ipynb"
%run $setup

In [ ]:
# program target with software AES
cw.program_target(scope, prog, "./simpleserial-aes-{}.hex".format(PLATFORM))

## Chip communication
### Goal:
1. send a message/key to be ciphered to the chip (protocol 'k' and 'p') and check it is indeed an AES cipher with the Python function
2. follow the protocol 'G' described in the text and cipher a plaintext with the unknown/secret key.
3. write the function `send_challenge(chip, plaintext)` which returns the encrypted plaintext. 


### Basic functions

In [ ]:
## sending a message to the chip 
## message is a byte array 
text = os.urandom(16)
target.simpleserial_write('g', text)
## receiving n octets from the device  
response = target.simpleserial_read('r', 16)
print("Response: " + response.hex())
## get random bytearray of length n
n = 16
rnd = os.urandom(n)
print(str(n) + " random bytes: " + rnd.hex())

### Cipher random plaintext with random key / chek results

In [ ]:
#set random key + plaintext
key = ...
plaintext = ...

# cipher with the chip
...
cipher_chip = ...

cipher_pyth = AES.new(key, AES.MODE_ECB).encrypt(plaintext)
print("Python:", cipher_pyth.hex())

print("Chip:  " , cipher_chip.hex())
assert(cipher_pyth.hex() ==  cipher_chip.hex() )

### Cipher with the secret key `g` + send_challenge function 

In [ ]:
secret_key = "00" * 16 # to be determined
cipher_pyth = AES.new(secret_key, AES.MODE_ECB).encrypt(plaintext)


cipher_chip = ...

print("Python:", cipher_pyth.hex())
print("Chip:  " , send_challenge(target, plaintext).hex())

In [ ]:
def send_challenge(target, plaintext):
    ...
    return cipher

## Scope 
#### Goal:
- discover and play with the scope, trigger, run stop, time/amplitude scale.
- Find the best position (~ more signal) to perform the attack 
- identify AES rounds => plot/save the corresponding curve
- identify parts of the first AES round => plot/save the corresponding trace
- record 2500 couples (trace, message) to be attacked
- (subsidiary) record 10000 characterization data (trace, message, key)

In [ ]:
# print scope parameters
scope

In [ ]:
print("Number of samples to be recorded: " + str(scope.adc.samples))
print("Decimation, for longer traces: " + str(scope.adc.decimate))

In [ ]:
# arm the scope to be ready for acquistions
scope.arm()
# the command 'p' includes a signal at the beginning of AES encryption that is sent to the scope to begin acquisition
target.simpleserial_write('g', text)
ret = scope.capture()
wave = scope.get_last_trace()
plt.plot(wave)
plt.show()

#### Plot interesting part of the AES

In [ ]:
# plot the whole AES and spot the rounds
scope.adc.samples = xxx
scope.adc.decimate = xxxx
# arm the scope to be ready for acquistions
scope.arm()
# the command 'p' includes a signal at the beginning of AES encryption that is sent to the scope to begin acquisition
target.simpleserial_write('g', text)
ret = scope.capture()

wave = scope.get_last_trace()
plt.plot(wave)
plt.show()

In [ ]:
# focus on AES first round
scope.adc.samples = xxxx
scope.adc.decimate = x
# arm the scope to be ready for acquistions
scope.arm()
# the command 'p' includes a signal at the beginning of AES encryption that is sent to the scope to begin acquisition
target.simpleserial_write('g', text)
ret = scope.capture()

wave = scope.get_last_trace()
plt.plot(wave)
plt.show()

#### Acquistion of traces/plaintexts
 -  acquire 2500 sets of (trace/plaintext). 

In [ ]:
def acquire_traces(target, count=10):
    '''target :
    count: number of curves to save
    returns plaintexts, traces
    '''    
    plaintexts = []
    # init to zero an array of 'count' traces each comprising 'scope.adc.samples' samples
    traces = np.zeros((count, scope.adc.samples), dtype=np.float)
    
    for i in tnrange(count, desc = "Capturing"):
        plaintext = os.urandom(16)
            
        #scope.stop()
    return traces, plaintexts

tr,pl = acquire_traces(target, 2500)

### Save traces and plaintexts in the AES folder

In [ ]:
dir_name = "./acqu"
np.save(dir_name + '/tracesAES', tr)
f = open(dir_name + "/plaintexts.txt", 'w')
for p in pl:
    f.write(str(p) + "\n")
f.close()